In [107]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd


In [108]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="ems"
    )

In [118]:
import pandas as pd
import mysql.connector



def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
       WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
valid_dates_vskp AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday)
),
valid_dates_ggm AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday_ggm)
),
attendance_vskp AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
attendance_ggm AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance_GGM
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
leave_days AS (
    -- First, expand leave periods into individual leave dates
    WITH RECURSIVE leave_expanded AS (
        SELECT 
            l.empname,
            e.work_location,
            GREATEST(l.`from`, DATE('2024-01-01')) AS leave_date,
            LEAST(l.`to`, DATE('2025-03-31')) AS leave_end
        FROM leaves l
        JOIN emp e ON e.empname = l.empname
        WHERE 
            l.status = 1
            AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
            AND l.`to` >= '2024-01-01'
            AND l.`from` <= '2025-03-31'
        
        UNION ALL
        
        SELECT 
            le.empname,
            le.work_location,
            le.leave_date + INTERVAL 1 DAY,
            le.leave_end
        FROM leave_expanded le
        WHERE le.leave_date + INTERVAL 1 DAY <= le.leave_end
    ),
    -- Now filter out Sundays and holidays based on location
    valid_leave_days AS (
        SELECT 
            le.empname,
            le.leave_date
        FROM leave_expanded le
        WHERE DAYOFWEEK(le.leave_date) != 1  -- Exclude Sundays
          AND (
              (le.work_location = 'Visakhapatnam' AND le.leave_date NOT IN (SELECT `date` FROM holiday)) OR
              (le.work_location = 'Gurugram' AND le.leave_date NOT IN (SELECT `date` FROM holiday_ggm))
          )
    )
    SELECT empname, COUNT(*) AS leave_days
    FROM valid_leave_days
    GROUP BY empname
)

,
present_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM attendance_vskp a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM attendance_ggm a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS biometric_present
    FROM emp e
),
working_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM valid_dates_vskp v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM valid_dates_ggm v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS total_working_days
    FROM emp e
)

SELECT 
    e.empname,
    e.empdoj,
    e.work_location,
    (COALESCE(p.biometric_present, 0) + COALESCE(l.leave_days, 0)) AS present_days,
    COALESCE(w.total_working_days, 0) AS working_days
FROM emp e
LEFT JOIN present_days p ON e.empname = p.empname
LEFT JOIN leave_days l ON e.empname = l.empname
LEFT JOIN working_days w ON e.empname = w.empname;

        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Convert the result into a DataFrame for easy handling
        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

# Fetch and display the results
df = first_feature()
df.head(25)


,empname,empdoj,work_location,present_days,working_days
0,BONDI MANOHAR,2023-09-11,Visakhapatnam,332,360
1,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,322,360
2,YATHIRAJAM SOWMYA,2022-10-01,Visakhapatnam,328,360
3,NAKKA ARAVIND REDDY,2022-12-01,Visakhapatnam,330,360
4,GORAPALLI MEGHA SHYAM,2022-12-05,Visakhapatnam,341,360
5,ABHILASH KUMAR SAHU,2023-03-01,Visakhapatnam,0,360
6,KUNDRAPU SRINIVAS,2023-07-11,Visakhapatnam,330,360
7,SURAMPUDI LAXMAN,2023-07-11,Visakhapatnam,332,360
8,INDALA SATISH KUMAR,2023-07-11,Visakhapatnam,330,360
9,GANDI MANOHARA RAO,2022-11-01,Visakhapatnam,122,360


In [120]:
df['empname'] = df['empname'].str.strip()
df_user_49 = df[df['empname'] == "PUTSALA HARSHA PRIYA"]
print(df_user_49)


                 empname      empdoj  work_location  present_days  \
47  PUTSALA HARSHA PRIYA  2024-06-04  Visakhapatnam           227   

    working_days  
47           236  


In [111]:
import pandas as pd
import mysql.connector


# Function to fetch present days
def fetch_present_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
        WITH RECURSIVE all_dates AS (
            SELECT DATE('2024-01-01') AS dt
            UNION ALL
            SELECT dt + INTERVAL 1 DAY
            FROM all_dates
            WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
        ),
        attendance_vskp AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        attendance_ggm AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        present_days AS (
            SELECT 
                e.empname,
                e.UserID,
                CASE 
                    WHEN e.work_location = 'Visakhapatnam' THEN (
                        SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                        FROM attendance_vskp a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    WHEN e.work_location = 'Gurugram' THEN (
                        SELECT GROUP_CONCAT(DISTINCT a.att_date ORDER BY a.att_date)
                        FROM attendance_ggm a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    ELSE ''
                END AS present_dates
            FROM emp e
            WHERE e.empname = 'GUJJI VARSHITHA'
        )
        SELECT 
            e.empname,
            e.UserID,
            p.present_dates
        FROM emp e
        LEFT JOIN present_days p ON e.UserID = p.UserID
        WHERE e.empname = 'GUJJI VARSHITHA';
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Extracting the present dates from the result
        if result:
            present_days = result[0]['present_dates']
            present_days_list = present_days.split(',') if present_days else []

            # Creating a DataFrame from the present days
            df = pd.DataFrame(present_days_list, columns=['Present Days'])

            # Saving the DataFrame to an Excel file
            excel_file_path = "./present_days_VARSHITHA.csv"
            df.to_csv(excel_file_path, index=False)

            return excel_file_path
        else:
            return "No data found for GUJJI VARSHITHA"

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return None

# Fetch the present days and generate the Excel file
file_path = fetch_present_days()
file_path


'./present_days_VARSHITHA.csv'

In [112]:
import pandas as pd
import mysql.connector


def fetch_present_days_with_debug():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        # Step 1: Biometric attendance dates
        biometric_query = """
        WITH attendance_vskp AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        attendance_ggm AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        emp_attendance_dates AS (
            SELECT 
                e.empname,
                a.att_date
            FROM emp e
            LEFT JOIN attendance_vskp a ON e.UserID = a.UserID
            WHERE e.work_location = 'Visakhapatnam'
              AND e.empname = 'PUTSALA HARSHA PRIYA'
              AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'

            UNION ALL

            SELECT 
                e.empname,
                a.att_date
            FROM emp e
            LEFT JOIN attendance_ggm a ON e.UserID = a.UserID
            WHERE e.work_location = 'Gurugram'
              AND e.empname = 'PUTSALA HARSHA PRIYA'
              AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
        )
        SELECT empname, att_date
        FROM emp_attendance_dates
        ORDER BY att_date;
        """

        cursor.execute(biometric_query)
        biometric_result = cursor.fetchall()
        biometric_df = pd.DataFrame(biometric_result)

        print(f"✅ Biometric present days count: {len(biometric_df)}")
        print(f"📅 First 5 biometric days:\n{biometric_df.head()}")

        # Step 2: Fetch leave days (WFH/OD/OL)
        leave_query = """
        SELECT 
            l.empname,
            DATE(l.`from`) AS leave_date
        FROM leaves l
        JOIN emp e ON l.empname = e.empname
        WHERE l.status = 1
          AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
          AND e.empname = 'PUTSALA HARSHA PRIYA'
          AND l.`from` BETWEEN '2024-01-01' AND '2025-03-31';
        """

        cursor.execute(leave_query)
        leave_result = cursor.fetchall()
        leave_df = pd.DataFrame(leave_result)

        print(f"✅ Leave days count (WFH/OD/OL): {len(leave_df)}")
        print(f"📅 First 5 leave days:\n{leave_df.head(10)}")

        # Combine both types
        biometric_df["type"] = "Biometric"
        leave_df["type"] = "Leave"

        combined_df = pd.concat([biometric_df.rename(columns={'att_date': 'date'}),
                                 leave_df.rename(columns={'leave_date': 'date'})])

        combined_df = combined_df.sort_values(by="date")

        print(f"📊 Total present days (combined): {len(combined_df)}")

        # Save to CSV
        excel_file_path = "./combined_present_days_PUTSALA HARSHA PRIYA.csv"
        combined_df.to_csv(excel_file_path, index=False)

        conn.close()
        return excel_file_path

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return None

# Run the debug-enhanced version
file_path = fetch_present_days_with_debug()
print("✅ File saved at:", file_path)


✅ Biometric present days count: 193
📅 First 5 biometric days:
                empname    att_date
0  PUTSALA HARSHA PRIYA  2024-06-04
1  PUTSALA HARSHA PRIYA  2024-06-05
2  PUTSALA HARSHA PRIYA  2024-06-06
3  PUTSALA HARSHA PRIYA  2024-06-07
4  PUTSALA HARSHA PRIYA  2024-06-08
✅ Leave days count (WFH/OD/OL): 5
📅 First 5 leave days:
                empname  leave_date
0  PUTSALA HARSHA PRIYA  2024-07-08
1  PUTSALA HARSHA PRIYA  2024-07-26
2  PUTSALA HARSHA PRIYA  2024-08-02
3  PUTSALA HARSHA PRIYA  2024-09-11
4  PUTSALA HARSHA PRIYA  2025-02-01
📊 Total present days (combined): 198
✅ File saved at: ./combined_present_days_PUTSALA HARSHA PRIYA.csv


In [126]:
import pandas as pd
import mysql.connector


def fetch_present_days_with_debug():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        # Step 1: Biometric attendance dates
        biometric_query = """
        WITH attendance_vskp AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        attendance_ggm AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        emp_attendance_dates AS (
            SELECT 
                e.empname,
                a.att_date
            FROM emp e
            LEFT JOIN attendance_vskp a ON e.UserID = a.UserID
            WHERE e.work_location = 'Visakhapatnam'
              AND e.empname = 'NARADA MOHAN NANDA KISHORE REDDY'
              AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'

            UNION ALL

            SELECT 
                e.empname,
                a.att_date
            FROM emp e
            LEFT JOIN attendance_ggm a ON e.UserID = a.UserID
            WHERE e.work_location = 'Gurugram'
              AND e.empname = 'NARADA MOHAN NANDA KISHORE REDDY'
              AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
        )
        SELECT empname, att_date
        FROM emp_attendance_dates
        ORDER BY att_date;
        """

        cursor.execute(biometric_query)
        biometric_result = cursor.fetchall()
        biometric_df = pd.DataFrame(biometric_result)

        print(f"✅ Biometric present days count: {len(biometric_df)}")
        print(f"📅 First 5 biometric days:\n{biometric_df.tail(5)}")

        # Step 2: Leave days
        leave_query = """
        WITH RECURSIVE leave_dates AS (
    -- Initial row: include first day (FROM date) only if it's not a Sunday or holiday
    SELECT 
        l.empname,
        e.work_location,
        DATE(l.`from`) AS leave_date,
        l.`to`
    FROM leaves l
    JOIN emp e ON l.empname = e.empname
    WHERE l.status = 1
      AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
      AND e.empname = 'NARADA MOHAN NANDA KISHORE REDDY'
      AND l.`to` >= '2024-01-01'
      AND l.`from` <= '2025-03-31'
      AND DAYOFWEEK(DATE(l.`from`)) != 1
      AND NOT EXISTS (
          SELECT 1 FROM holiday h
          WHERE h.date = DATE(l.`from`)
            AND h.status = 1
            AND (
                (e.work_location = 'Gurugram' AND h.value = 'GGM') OR
                (e.work_location = 'Visakhapatnam' AND h.value = 'VSKP')
            )
      )

    UNION ALL

    -- Expand one day at a time
    SELECT 
        ld.empname,
        ld.work_location,
        DATE_ADD(ld.leave_date, INTERVAL 1 DAY),
        ld.`to`
    FROM leave_dates ld
    WHERE DATE_ADD(ld.leave_date, INTERVAL 1 DAY) <= ld.`to`
      AND DAYOFWEEK(DATE_ADD(ld.leave_date, INTERVAL 1 DAY)) != 1
      AND NOT EXISTS (
          SELECT 1 FROM holiday h
          WHERE h.date = DATE_ADD(ld.leave_date, INTERVAL 1 DAY)
            AND h.status = 1
            AND (
                (ld.work_location = 'Gurugram' AND h.value = 'GGM') OR
                (ld.work_location = 'Visakhapatnam' AND h.value = 'VSKP')
            )
      )
)
SELECT empname, leave_date
FROM leave_dates
ORDER BY leave_date;

        """

        cursor.execute(leave_query)
        leave_result = cursor.fetchall()
        leave_df = pd.DataFrame(leave_result)

        print(f"✅ Leave days count (WFH/OD/OL): {len(leave_df)}")
        print(f"📅 First 10 leave days:\n{leave_df.head(15)}")

        # Combine both
        biometric_df["type"] = "Biometric"
        leave_df["type"] = "Leave"

        combined_df = pd.concat([
            biometric_df.rename(columns={'att_date': 'date'}),
            leave_df.rename(columns={'leave_date': 'date'})
        ])

        combined_df = combined_df.sort_values(by="date").reset_index(drop=True)

        print(f"📊 Total present days (combined): {len(combined_df)}")

        # Save to CSV
        excel_file_path = "./combined_present_days_PUTSALA_HARSHA_PRIYA.csv"
        combined_df.to_csv(excel_file_path, index=False)

        conn.close()
        return excel_file_path

    except mysql.connector.Error as err:
        print(f"❌ Database error: {err}")
        return None


# Run
file_path = fetch_present_days_with_debug()
print("✅ File saved at:", file_path)


✅ Biometric present days count: 339
📅 First 5 biometric days:
                              empname    att_date
334  NARADA MOHAN NANDA KISHORE REDDY  2025-03-25
335  NARADA MOHAN NANDA KISHORE REDDY  2025-03-26
336  NARADA MOHAN NANDA KISHORE REDDY  2025-03-27
337  NARADA MOHAN NANDA KISHORE REDDY  2025-03-28
338  NARADA MOHAN NANDA KISHORE REDDY  2025-03-29
✅ Leave days count (WFH/OD/OL): 4
📅 First 10 leave days:
                            empname  leave_date
0  NARADA MOHAN NANDA KISHORE REDDY  2024-09-11
1  NARADA MOHAN NANDA KISHORE REDDY  2024-09-12
2  NARADA MOHAN NANDA KISHORE REDDY  2024-09-13
3  NARADA MOHAN NANDA KISHORE REDDY  2024-09-14
📊 Total present days (combined): 343
✅ File saved at: ./combined_present_days_PUTSALA_HARSHA_PRIYA.csv


In [ ]:
# SELECT * 
# FROM `leaves`
# WHERE leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
#   AND empname = 'PUTSALA HARSHA PRIYA';
